In [13]:
from ph import LexicalURLFeature
from ph2 import ContentFeatures
from ph3 import HostFeatures
import whois
import pandas as pd

In [3]:
data0 = pd.read_csv("online-valid.csv")
data0.head()

,phish_id,url,phish_detail_url,submission_time,verified,verification_time,online,target
0,8030352,https://2.alexandraaparta.repl.co/,http://www.phishtank.com/phish_detail.php?phis...,2023-02-08T17:22:20+00:00,yes,2023-02-08T17:32:25+00:00,yes,Other
1,8030353,https://esim-demande.fr/verification/compte/lo...,http://www.phishtank.com/phish_detail.php?phis...,2023-02-08T17:22:20+00:00,yes,2023-02-08T17:32:25+00:00,yes,Other
2,8030350,https://audiotekx.com/Cold/1/index.php,http://www.phishtank.com/phish_detail.php?phis...,2023-02-08T17:15:21+00:00,yes,2023-02-08T17:32:25+00:00,yes,Internal Revenue Service
3,8030348,https://323.alexandraaparta.repl.co/,http://www.phishtank.com/phish_detail.php?phis...,2023-02-08T17:12:18+00:00,yes,2023-02-08T17:32:25+00:00,yes,Other
4,8030331,https://masksi.webcindario.com/home.php,http://www.phishtank.com/phish_detail.php?phis...,2023-02-08T17:01:04+00:00,yes,2023-02-08T17:32:25+00:00,yes,Other


In [5]:
data0.shape

(20745, 8)

In [6]:
phishurl = data0.sample(n = 1000, random_state = 12).copy()
phishurl = phishurl.reset_index(drop=True)
phishurl.head()

,phish_id,url,phish_detail_url,submission_time,verified,verification_time,online,target
0,7735190,https://storageapi.fleek.co/b620e7da-f4f5-4e6e...,http://www.phishtank.com/phish_detail.php?phis...,2022-09-07T22:34:19+00:00,yes,2022-09-07T22:41:55+00:00,yes,Other
1,7990204,https://mingovplkpkx.dioturnpestsi.ml,http://www.phishtank.com/phish_detail.php?phis...,2022-12-30T13:19:29+00:00,yes,2022-12-30T13:22:46+00:00,yes,Other
2,7994781,https://att-105146.weeblysite.com/,http://www.phishtank.com/phish_detail.php?phis...,2023-01-03T02:34:43+00:00,yes,2023-01-03T03:09:30+00:00,yes,Other
3,7503957,https://webservice-mailupdatemail.arqanexportc...,http://www.phishtank.com/phish_detail.php?phis...,2022-05-01T22:41:11+00:00,yes,2022-05-01T22:52:46+00:00,yes,Other
4,8009782,https://voyons-ensemble-lavenir.firebaseapp.com/,http://www.phishtank.com/phish_detail.php?phis...,2023-01-18T21:13:29+00:00,yes,2023-01-18T21:32:18+00:00,yes,Other


In [7]:
data1 = pd.read_csv("Benign_list_big_final.csv")
data1.columns = ['URLs']
data1.head()

,URLs
0,http://1337x.to/torrent/1110018/Blackhat-2015-...
1,http://1337x.to/torrent/1122940/Blackhat-2015-...
2,http://1337x.to/torrent/1124395/Fast-and-Furio...
3,http://1337x.to/torrent/1145504/Avengers-Age-o...
4,http://1337x.to/torrent/1160078/Avengers-age-o...


In [8]:
#Collecting 1,000 Legitimate URLs randomly
legiurl = data1.sample(n = 1000, random_state = 12).copy()
legiurl = legiurl.reset_index(drop=True)
legiurl.head()

,URLs
0,http://graphicriver.net/search?date=this-month...
1,http://ecnavi.jp/redirect/?url=http://www.cros...
2,https://hubpages.com/signin?explain=follow+Hub...
3,http://extratorrent.cc/torrent/4190536/AOMEI+B...
4,http://icicibank.com/Personal-Banking/offers/o...


In [9]:
feat_names = ['UrlLength','PathLength','Hostlength','HostIsIp','PortInUrl','NumDigits','NumQueryParams',
            'NumFragments','IsEncoded','NumEncodings','NumSubDirs','NumPeriods','HasClient','HasAdmin',
            'HasServer','HasLogin','HasAtsign','Redirection','UrlShortened','UsesHttps','DashInDomain',
            'HtmlLen','NumHtmlTags','NumHiddenTags','NumScriptTags','NumIframes','NumEmbeds','NumHyperLinks',
            'NumEvalFuncs','WebForwards', 'DomainAge', 'DomainIntendedLife_Span','LifeRemaining',
            'IsLive','ConnectionSpeed','AvgUpdateFrequency','NumUpdates','TTL', "ClassLabel"]
def feat_extraction(url, label):
    UrlFeats = LexicalURLFeature(url)
    ContFeats = ContentFeatures(url)
    HostFeats = HostFeatures(url)

    feats = []
    ph_methods = ['url_length','url_path_length','url_host_length','url_host_is_ip','url_contains_port','num_of_digits',
                'num_of_parameters','num_of_fragments','is_encoded','num_encoded_chars','num_of_subdirectories',
                'num_of_periods','has_client_in_string','has_admin_in_string','has_server_in_string',
                'has_login_in_string','has_at_sign','redirection','tiny_url','uses_https','dash_in_domain']
    ph2_methods = ['length_of_html','number_of_html_tags','number_of_hidden_tags','number_of_script_tags','number_iframes',
                'number_embeds','number_of_hyperlinks','number_of_eval_functions','forwarding']
    ph3_methods = ['url_age','url_intended_life_span','url_life_remaining','url_is_live','url_connection_speed',
                'average_update_frequency','number_of_updates','ttl_from_registration']
    for method in ph_methods:
        feats.append(getattr(UrlFeats, method)())  # call
    for method in ph2_methods:
        feats.append(getattr(ContFeats, method)())  # call
    for method in ph3_methods:
        feats.append(getattr(HostFeats, method)())  # call
    feats.append(label)

In [14]:
#Extracting the feautres & storing them in a list
legi_features = []
label = 0

for i in range(0, legiurl.shape[0]):
  url = legiurl['URLs'][i]
  legi_features.append(feat_extraction(url,label))


ReadTimeout: HTTPConnectionPool(host='extratorrent.cc', port=80): Read timed out. (read timeout=5)

In [ ]:
#Extracting the feautres & storing them in a list
phish_features = []
label = 1
for i in range(0, phishurl.shape[0]):
  url = phishurl['url'][i]
  phish_features.append(feat_extraction(url,label))

In [ ]:
all_feats = legi_features + phish_features
dataframe = pd.DataFrame(all_feats, columns=feat_names)
dataframe.head

In [ ]:
dataframe.to_csv('phishing_dataset.csv', index= False)